# Analyzing Data in a ROOT Tree
## Why Trees?

ROOT trees are optimized for the <span style="color:red">storage</span> of the kind of data produced in high-energy and nuclear physics experiments:

* Very large numbers of _events_ with essentially the same data structure
* Variable length containers holding identical data/objects
* Tree-like structures of collections of objects, similar to databases

<span style="color:red">Access patterns</span> to data in ROOT trees are also optimized for the kinds we typically need for our analyses:

* "Column-wise" reading of individual data elements—only the element(s) of interest are read, not entire events
* Only one event (or part of it) in memory at a time (modulo buffer size)
* Buffered to disk, some degree of integrity protection during writing ("cycles")

Without structures like ROOT trees, the efficient analysis of petabyte-size data sets from CERN and elsewhere would be nearly impossible. ROOT trees

* minimize memory requirements and I/O
* greatly enhance access speed

In addition to simple n-tuple-like data, ROOT <span style="color:red">lets you write C++ objects to file</span>. This is impossible in native C++ and is achieved through class dictionaries generated with the _Cling_ C++ interpreter/compiler. The ability to write <span style="color:red">entire object trees</span> is critical for storing the very complex event data from HEP experiments. This is also knows as "C++ object persistency". Some programming languages offer object persistency natively, but not C++.

## Hall A Analyzer trees

ROOT trees produced by the Hall A and Hall C analyzers have a rather simple structure. Each "global variable" (analysis result) selected for output to the tree is written to its own branch with only a single leaf. The name of the branch is identical to the name of the global variable, for example ```L.tr.p```. At this time, data branches always have the type ```Double_t```. We are planning to support other data types, in particular integers, starting with analyzer version 1.7.

In the case of arrays, a second branch is written to the tree whose name is ```Ndata.``` followed by the name of the corresponding array, _e.g._ ```Ndata.L.tr.p```. These branches always have type ```Int_t``` and hold the number of elements in the array for the current event. There is a lot of redundancy in ```Ndata``` variables for various elements of a single object, such as the collection of tracks. We may reduce or even eliminate this redundancy in a future version of the analyzer.

Here is an example of a typical Hall A analyzer output tree where all the Left-HRS track data have been written out with a ```block L.tr.*``` statement in the output defintion file:

![TreeView](img/TreeView3.png)

To explore this exact tree yourself, start an interactive ```analyzer``` or ROOT session and type:
```
analyzer [1] f = TFile::Open("/data/ROOTfiles/g2p_3132.root","READ");
analyzer [2] b = new TBrowser;
```
or, if you didn't download the file yet,
```
analyzer [1] f = TFile::Open("http://hallaweb.jlab.org/data_reduc/AnaWork2018/ROOTfiles/g2p_3132.root");
analyzer [2] b = new TBrowser;
```

## How to work with tree data
### Inspecting text output

ROOT offers a number of ways to work with data in trees. First, there are two commands that produce text output. They are essentially legacy commands from the days of PAW, but nevertheless they work well with the n-tuples in our trees:

<div style="background: #d9edf7; border-color: #bce8f1; border-bottom: 5px solid #bce8f1; color: #31708f;  padding: 15px; margin-top: 20px; margin-bottom: 20px">
<ul>
<li> __Scan__: Prints a table where each row corresponds to an event and and each column, to the branch data. If there are multiple entries in a variable-sized array, multiple rows are printed for a single event, each row corresponding to the array index, called "Instance". Allows quick comparison of a number of columns (often faster and clearer than plotting).
<li> __Show__: Prints all data for a single event. The output can be large. Helps with understanding the data structure. Often used for inspecting unusual events such as misreconstructed tracks.
</ul></div>

Let's try these

**TTree::Scan**

In [ ]:
// Open ROOT file
f = TFile::Open("/data/ROOTfiles/g2p_3132.root","READ");

Ignore the warnings about ```"no dictionary for class ..."``` These occur because we are running plain ROOT and not the ```analyzer```.

In [ ]:
// Look at the contents of the file
f->ls();

Note the tree named __T__

In [ ]:
// Print the value of the "L.tr.p" variable (momentum of reconstructed track in GeV)
// and "L.tr.vz" (vertex z-coordinate in m) for the first 10 events. 
T->Scan("L.tr.p:L.tr.vz","","",10);

Let's scan again, this time selecting events with multiple tracks. Because these are rare (at the level of a few percent), let's scan the first 500 events.

In [ ]:
T->Scan("L.tr.p:L.tr.vz","L.tr.n>1","",500);

Modify the command above to scan the first 1000 events or more. Once the output fills a good screenful, ROOT will prompt you if you wish to continue or quit. (The prompt is ignored in the notebook environment.)

As you can see, there are now multiple instances per event (=row number). The ```L.tr.p``` and ```L.tr.vz``` arrays are parallel, _i.e._ for both arrays the index has the same meaning. If you are unsure if arrays are parallel, you can plot the corresponding ```Ndata``` elements against each other. To do so, we can use ```T->Draw()```, which we'll discuss a little later.

**TTree::Show**

The second text-based command to inspect trees is ```TTree::Show(entry)```. It simply prints the entire contents of the given entry (event). Let's give it a try:

In [ ]:
T->Show(5);

As you can guess, this is mostly useful for understanding odd outlier-type events. The output can be very lengthy and is often best redirected to a file using ```.> filename``` at the ROOT command prompt. 

You may notice that a few values in this event are set to ```1e+38```. This is a special value, ```kBig```, that the analyzer uses to indicate that this value was never set. In the above analysis, no beta calculation was done, so the corresponding data equal ```kBig```. While this is convenient for distinguishing, say, a zero reading from the "never read" case, unfortunately this convention also makes it difficult to plot any tree variables with at least a trivial cut like ```abs(L.tr.x)<10``` because there will always be some events that weren't fully computed, e.g. because of missing track data etc.

### Graphical & Programmatic Processing

Here is a summary of the common methods for graphical and programmatic analysis of trees.

<div style="background: #d9edf7; border-color: #bce8f1; border-bottom: 5px solid #bce8f1; color: #31708f;  padding: 15px; margin-top: 20px; margin-bottom: 20px">
<ul>
<li> __Draw__: Extremely useful command for plotting tree data. Produces 1-D and 2-D histograms as well as 3-D graphs with various selection and drawing options.
<li> __MakeClass__: Legacy function to create skeleton C++ code for a _specific_ TTree. An event loop is provided for you in the source file, which you need to fill in. 
<li> __MakeSelector__: Improved method to create skeleton C++ code. Provides a processing class deriving from TSelector with various callback functions. ROOT does the looping over events for you, and you have to specify what you would like to do for each event. Compared to MakeClass code, it is better organized, faster, and suitable for multithreading. Available as of ROOT 6. 
<li> __TDataFrame__: Experimental processing framework featuring implicit multithreading and on-demand execution. Relies heavily on C++11. Currently still quite buggy and slow. Available as of ROOT 6.10 with a mainstream appearance in ROOT 6.14/00. Essentially a preview of what to expect in ROOT 7.
</ul></div>

In the following two sections, we will walk through examples of ```TTree::Draw``` and ```TTree::MakeSelector```. Click on "Next Section" below.

<p style="position:relative; width:95%; float:left; text-align:right;"><a href="Analyzing-Trees-Section2.ipynb">Next: Section 2, TTree::Draw &gt;&gt;</a></p>